### Install all the neccessaary package

In [1]:
%pip install numpy
%pip install opencv-python
%pip install opencv-contrib-python

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Import packages and libraries

In [2]:
# A computer vision library
import cv2
# A scientific computation library
import numpy as np

### Test a camera

In [3]:

vid = cv2.VideoCapture(0)

while True:
    ret, frame = vid.read()
    
    # Add text at bottom right
    cv2.putText(frame, "Click 'q' to exit webcam", (frame.shape[1] - 300, frame.shape[0] - 20), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    cv2.imshow("Video", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

2025-07-10 14:01:30.954 Python[9814:91813] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


AttributeError: 'NoneType' object has no attribute 'shape'

### Task One:

Instruct students to read videos stored on the computer

### Face Detection

This is to be able to draw a bounding box around the face.

We are also gathering data for the training of the recogition. The code will request for a name, put a name in the box provided then press Enter. A window frame will pop up, and when it sees a face it will draw a green bounding as it has detected a face. When the bounding box is not shaking and it is stable, press q and you will be prompted for a name again. The process will be repeated all over again. We set training for five(5) people

In [13]:
cap = cv2.VideoCapture(0)
face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
face_data = []
labels = []
number_of_people = 2
for i in range(number_of_people):
    name = input(f"Enter the name of person {i+1}: ")
    while True:
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
        for (x, y, w, h) in faces:
            face = gray[y:y + h, x:x + w]
            face = cv2.resize(face, (512, 512), interpolation=cv2.INTER_AREA)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 4)
            cv2.putText(frame, "Click 'q' to save image", (frame.shape[1] - 300, frame.shape[0] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.imshow('Frame', frame)

            if 0xFF == ord('q'):
                break

        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    face_data.append(face)
    labels.append(name)
cap.release()
cv2.destroyAllWindows()


# Train on different people both through webcam and videos

### Train a face recognition model

In [16]:
# Create a LBPH face recognizer
labels2={i:j for i,j in enumerate(labels)}

print(labels2, face_data)
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

# Train the face recognizer on the dataset
face_recognizer.train(face_data, np.array(list(labels2.keys())))

{0: 'Sadeeq', 1: 'Aliyu'} [array([[209, 205, 202, ..., 206, 217, 194],
       [208, 202, 200, ..., 206, 217, 197],
       [208, 202, 199, ..., 207, 215, 199],
       ...,
       [ 72,  73,  73, ..., 185, 219, 198],
       [ 73,  73,  73, ..., 186, 220, 196],
       [ 73,  73,  73, ..., 185, 223, 192]], shape=(512, 512), dtype=uint8), array([[222, 189, 203, ..., 223, 169, 220],
       [223, 189, 204, ..., 224, 167, 223],
       [224, 187, 206, ..., 225, 165, 225],
       ...,
       [194, 194, 194, ..., 199, 169, 236],
       [193, 193, 193, ..., 197, 167, 232],
       [193, 193, 193, ..., 199, 173, 232]], shape=(512, 512), dtype=uint8)]


### Face Recognition:

This part of the code is to be able to detect the person that was stored. The code will detect the face of anyone stored and it will be unknown if the person is not part of the data. Press q when done.

In [15]:

# Real-time face recognition from webcam
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_roi = gray[y:y+h, x:x+w]
        face_features = cv2.resize(face, (512, 512), interpolation=cv2.INTER_AREA)
        label, confidence = face_recognizer.predict(face_features)
        print(confidence)
        detection_name = labels2[label] if confidence > 0.6 else "Unknown"
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, str(labels2[label]), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

    cv2.imshow('Face Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# Task on how to save models using cv.save()

In [ ]:
# Take home task, create a facial authentication system on streamlit

# Register face(s): Sign in
# Login using Face registered